# Semantisk Kjerne med OpenBnB MCP Server-integrasjon

Denne notatboken viser hvordan du kan bruke Semantisk Kjerne med den faktiske OpenBnB MCP-serveren for å søke etter ekte Airbnb-overnattingssteder ved hjelp av MCPStdioPlugin. For tilgang til LLM brukes Azure AI Foundry. For å sette opp miljøvariablene dine, kan du følge [Oppsettsleksjonen](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Opprette MCP Plugin-tilkoblingen

Vi skal koble til [OpenBnB MCP-serveren](https://github.com/openbnb-org/mcp-server-airbnb) ved hjelp av MCPStdioPlugin. Denne serveren tilbyr søkefunksjonalitet for Airbnb gjennom @openbnb/mcp-server-airbnb-pakken.


## Opprette klienten

I dette eksemplet skal vi bruke Azure AI Foundry for tilgang til LLM. Sørg for at miljøvariablene dine er riktig konfigurert.


## Miljøkonfigurasjon

Konfigurer Azure OpenAI-innstillinger. Sørg for at du har følgende miljøvariabler satt:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Forstå OpenBnB MCP-integrasjonen

Denne notatboken kobler til **den ekte OpenBnB MCP-serveren** som gir faktisk Airbnb-søkefunksjonalitet.

### Hvordan det fungerer:

1. **MCPStdioPlugin**: Bruker standard inn-/utgangskommunikasjon med MCP-serveren
2. **Ekte NPM-pakke**: Laster ned og kjører `@openbnb/mcp-server-airbnb` via npx
3. **Live-data**: Returnerer faktisk Airbnb-eiendomsdata fra deres API-er
4. **Funksjonsoppdagelse**: Agenten oppdager automatisk tilgjengelige funksjoner fra MCP-serveren

### Tilgjengelige funksjoner:

OpenBnB MCP-serveren tilbyr vanligvis:
- **search_listings** - Søk etter Airbnb-eiendommer basert på sted og kriterier
- **get_listing_details** - Få detaljert informasjon om spesifikke eiendommer
- **check_availability** - Sjekk tilgjengelighet for spesifikke datoer
- **get_reviews** - Hent anmeldelser for eiendommer
- **get_host_info** - Få informasjon om eiendomsverter

### Forutsetninger:

- **Node.js** installert på systemet ditt
- **Internettforbindelse** for å laste ned MCP-serverpakken
- **NPX** tilgjengelig (følger med Node.js)

### Testing av tilkoblingen:

Du kan teste MCP-serveren manuelt ved å kjøre:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Dette vil laste ned og starte OpenBnB MCP-serveren, som Semantic Kernel deretter kobler til for ekte Airbnb-data.


## Kjøre agenten med OpenBnB MCP-serveren

Nå skal vi kjøre AI-agenten som kobler seg til OpenBnB MCP-serveren for å søke etter ekte Airbnb-overnattingssteder i Stockholm for 2 voksne og 1 barn. Du kan gjerne endre `user_inputs`-listen for å tilpasse søkekriteriene.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Sammendrag  
Gratulerer! Du har med suksess bygget en AI-agent som integrerer med søk etter overnatting i den virkelige verden ved hjelp av Model Context Protocol (MCP):

Teknologier brukt:  
- Semantic Kernel - For å bygge intelligente agenter med Azure OpenAI  
- Azure AI Foundry - For LLM-funksjonalitet og chat-komplettering  
- MCP (Model Context Protocol) - For standardisert verktøyintegrasjon  
- OpenBnB MCP Server - For ekte Airbnb-søkefunksjonalitet  
- Node.js/NPX - For å kjøre den eksterne MCP-serveren  

Hva du har lært:  
- MCP-integrasjon: Koble Semantic Kernel-agenter til eksterne MCP-servere  
- Sanntidsdata-tilgang: Søke etter faktiske Airbnb-eiendommer via live API-er  
- Protokollkommunikasjon: Bruke stdio-kommunikasjon mellom agent og MCP-server  
- Funksjonsoppdagelse: Automatisk oppdage tilgjengelige funksjoner fra MCP-servere  
- Strømming av svar: Fange opp og logge funksjonskall i sanntid  
- HTML-rendering: Formatere agentens svar med stiliserte tabeller og interaktive visninger  

Neste steg:  
- Integrere flere MCP-servere (vær, fly, restauranter)  
- Bygge et multi-agent-system som kombinerer MCP- og A2A-protokoller  
- Lage tilpassede MCP-servere for dine egne datakilder  
- Implementere vedvarende samtaleminne på tvers av økter  
- Distribuere agenten til Azure Functions med MCP-server-orkestrering  
- Legge til brukergodkjenning og bestillingsfunksjonalitet  

Viktige fordeler med MCP-arkitektur:  
- Standardisering: Universell protokoll for å koble AI-agenter til eksterne verktøy  
- Sanntidsdata: Tilgang til live, oppdatert informasjon fra ulike tjenester  
- Utvidbarhet: Enkel integrasjon av nye datakilder og verktøy  
- Interoperabilitet: Fungerer på tvers av ulike AI-rammeverk og agentplattformer  
- Ansvarsdeling: Klar distinksjon mellom AI-logikk og ekstern data-tilgang  



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
